In [1]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# 演習用のコンテンツを取得
# !git clone https://github.com/matsuolab/lecture-ai-engineering.git
!git clone https://github.com/kazukitakayamas/llm-engineering-RAG.git

Cloning into 'llm-engineering-RAG'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 23 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 37.43 KiB | 618.00 KiB/s, done.
Resolving deltas: 100% (5/5), done.


In [3]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import random
random.seed(0)

In [5]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [6]:
def generate_output(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [7]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "Deepseekショックとは、何かについて分かりやすく教えて。"
response = generate_output(question, system_prompt)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpe

In [8]:
print(response)

Deepseek Shockは、AI-poweredの音声認識技術です。主に、音声データを分析し、音声の特徴やパターンを抽出することで、音声の内容や意味を理解することを目指しています。

Deepseek Shockは、音声認識技術の分野で活躍するAIの1つであり、音声データを分析し、音声の特徴やパターンを抽出することで、音声の内容や意味を理解することを目指しています。例えば、音声認識技術を用いて、会話の内容を理解し、感情や意図を分析することができます。

また、Deepseek Shockは、音声認識技術を応用して、音声の分類や検出、音声の生成や合成など、音声に関する多くのアプリケーションに使用されます。例えば、音声認識技術を用いて、音声の分類や検出を行うことで、音声の内容や意味を理解し、音声の生成や合成を行うことができます。

Deepseek Shockは、音声認識技術の分


- 数値的な評価も見てみます。RagasにはAnswer Accuracyという評価指標があります。今回はこちらを参考に実装した評価関数を利用して測っていきます。


In [9]:
!pip install -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.0/662.0 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.0
    Uninstalling openai-1.76.0:
      Successfully uninstalled openai-1.76.0


In [8]:
# # @title 5点満点評価実装
# gold_answer = "DeepSeekショックとは、中国浙江省杭州市を拠点とするAI開発企業が提供する次世代AI推論エンジン「DeepSeek」の登場により引き起こされた、業界全体の急激な変化とインパクトを指します。この技術は、膨大なデータを短時間で解析し、適切な予測や意思決定を支援する能力を持つことで、従来の業務や市場構造に劇的な影響を与えています。DeepSeekの3つの特徴超高速なデータ処理 DeepSeekは、大量のデータをリアルタイムで解析する能力を持ち、迅速な意思決定を可能にします。汎用性の高さ 金融、医療、製造、マーケティングなど、多様な分野に応用可能な柔軟性を持っています。操作のシンプルさ 高度な技術がバックエンドに存在しながら、直感的で使いやすいインターフェースを提供。"

# from openai import OpenAI
# from google.colab import userdata
# client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

# def openai_generator(query):
#         messages = [
#                     {
#                         "role": "user",
#                         "content": query
#                     }
#                 ]

#         response = client.chat.completions.create(
#             model="gpt-4o-mini",
#             messages=messages
#         )
#         return response.choices[0].message.content

# def evaluate_answer_accuracy(query, response, reference):

#     template_accuracy1 = (
#           "Instruction: You are a world class state of the art assistant for rating "
#           "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
#           "Rate the User Answer on a scale of 0 to 5 based on how well it matches the Reference Answer in content and accuracy:\n"
#           "- 0: The User Answer completely misses the point or does not relate to the question at all.\n"
#           "- 1: The User Answer has a small portion that aligns with the Reference Answer, but most of it is off-topic or incorrect.\n"
#           "- 2: The User Answer partially aligns with the Reference Answer, but around half of it deviates from the correct content.\n"
#           "- 3: The User Answer mostly aligns with the Reference Answer, but still has notable deviations or omissions.\n"
#           "- 4: The User Answer largely matches the Reference Answer with only minor inconsistencies or omissions.\n"
#           "- 5: The User Answer completely or mostly matches the Reference Answer in all important aspects and nuances.\n"
#           "Do not explain or justify your rating. Your rating must be only a single number (0, 1, 2, 3, 4, or 5) according to the instructions above.\n"
#           "### Question: {query}\n"
#           "### {answer0}: {sentence_inference}\n"
#           "### {answer1}: {sentence_true}\n"
#           "The rating is:\n"
#       )
#     template_accuracy2 = (
#           "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
#           "I will use a 0-5 scale where:\n"
#           "- 0: The User Answer completely misses the point or contains incorrect information.\n"
#           "- 1: The User Answer has some valid points but mostly deviates from the Reference Answer.\n"
#           "- 2: The User Answer aligns with the Reference Answer in about half of its content, but has significant deviations.\n"
#           "- 3: The User Answer partially aligns with the Reference Answer, with some inconsistencies or missing elements.\n"
#           "- 4: The User Answer largely aligns with the Reference Answer with only minor inconsistencies.\n"
#           "- 5: The User Answer is entirely or mostly consistent with the Reference Answer in all key aspects.\n"
#           "I will provide the rating without any explanation or justification, adhering strictly to this 0-5 scale.\n"
#           "Do not explain or justify my rating. My rating must be only a single number (0, 1, 2, 3, 4, or 5).\n\n"
#           "Question: {query}\n\n"
#           "{answer0}: {sentence_inference}\n\n"
#           "{answer1}: {sentence_true}\n\n"
#           "Rating: "
#       )

#     score1 = openai_generator(
#                 template_accuracy1.format(
#                       query=query,
#                       answer0="User Answer",
#                       answer1="Reference Answer",
#                       sentence_inference=response,
#                       sentence_true=reference,
#                     )
#                 )
#     try:
#       score1 = int(score1)
#     except:
#       print("Failed to parse score1")
#       score1 = 0

#     score2 = openai_generator(
#                 template_accuracy2.format(
#                         query=query,
#                         answer0="Reference Answer",
#                         answer1="User Answer",
#                         sentence_inference=reference,
#                         sentence_true=response,
#                     )
#                   )

#     try:
#       score2 = int(score2)
#     except:
#       print("Failed to parse score2")
#       score2 = 0

#     return (score1 + score2) / 2

In [12]:
# @title 5点満点評価実装

from openai import OpenAI
from google.colab import userdata
import re
import time

gold_answer = "DeepSeekショックとは、中国浙江省杭州市を拠点とするAI開発企業が提供する次世代AI推論エンジン「DeepSeek」の登場により引き起こされた、業界全体の急激な変化とインパクトを指します。この技術は、膨大なデータを短時間で解析し、適切な予測や意思決定を支援する能力を持つことで、従来の業務や市場構造に劇的な影響を与えています。DeepSeekの3つの特徴超高速なデータ処理 DeepSeekは、大量のデータをリアルタイムで解析する能力を持ち、迅速な意思決定を可能にします。汎用性の高さ 金融、医療、製造、マーケティングなど、多様な分野に応用可能な柔軟性を持っています。操作のシンプルさ 高度な技術がバックエンドに存在しながら、直感的で使いやすいインターフェースを提供。"

client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

def openai_generator(query, retry_attempts=3, delay=2):
    """拡張されたOpenAI API呼び出し関数"""
    for attempt in range(retry_attempts):
        try:
            messages = [
                {
                    "role": "user",
                    "content": query
                }
            ]

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                temperature=0.1  # より一貫性のある回答のために低温度を設定
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"API呼び出し失敗 (試行 {attempt+1}/{retry_attempts}): {e}")
            if attempt < retry_attempts - 1:
                time.sleep(delay)  # 次の試行前に待機

    print("すべてのAPI呼び出し試行が失敗しました")
    return None

def extract_numeric_score(text):
    """テキストから数値スコアを抽出する補助関数"""
    if not text:
        return 0

    # 単一の数字（0-5）を探す
    matches = re.findall(r'\b[0-5]\b', text)
    if matches:
        return int(matches[-1])  # 最後に見つかった数字を使用（結論が最後に来ることが多い）

    # 文字で書かれた数字も探す
    number_words = {
        'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5,
        '０': 0, '１': 1, '２': 2, '３': 3, '４': 4, '５': 5  # 全角数字
    }

    for word, value in number_words.items():
        if word in text.lower():
            return value

    return 0  # デフォルト値

def evaluate_answer_accuracy(query, response, reference):
    """元の関数シグネチャを維持しながら内部実装を改善"""

    # 改善されたテンプレート1：より明確な日本語の評価基準を含む
    template_accuracy1 = (
        "以下の質問と回答を分析してください。\n"
        "ユーザー回答が基準回答の内容とどれだけ一致しているかを0から5の尺度で評価してください：\n"
        "- 0: ユーザー回答は質問の要点を完全に外しているか、全く関連がない\n"
        "- 1: ユーザー回答には基準回答と一致する小さな部分があるが、大部分は的外れまたは不正確\n"
        "- 2: ユーザー回答は基準回答と部分的に一致するが、約半分の内容が基準から逸脱している\n"
        "- 3: ユーザー回答はおおむね基準回答と一致するが、顕著な逸脱や省略がある\n"
        "- 4: ユーザー回答は基準回答とほぼ一致しており、ごくわずかな不一致や省略がある\n"
        "- 5: ユーザー回答は基準回答とすべての重要な側面で完全に一致している\n"
        "あなたの評価は数字（0、1、2、3、4、または5）のみを回答してください。説明は不要です。\n\n"
        "質問: {query}\n"
        "ユーザー回答: {sentence_inference}\n"
        "基準回答: {sentence_true}\n"
        "評価:"
    )

    # 改善されたテンプレート2：評価基準を明確にし、逆順での評価
    template_accuracy2 = (
        "あなたは質問への回答を評価する専門家です。\n"
        "ユーザー回答が基準回答と比較してどれだけ正確かを0〜5のスケールで評価してください：\n"
        "- 0: ユーザー回答は完全に不正確か質問と無関係\n"
        "- 1: ユーザー回答には正しい点もあるが、ほとんどは基準回答から逸脱している\n"
        "- 2: ユーザー回答は内容の約半分が基準回答と一致するが、重大な逸脱がある\n"
        "- 3: ユーザー回答は基準回答と部分的に一致するが、一部の矛盾や欠落要素がある\n"
        "- 4: ユーザー回答は軽微な不一致を除いて基準回答とほぼ一致している\n"
        "- 5: ユーザー回答はすべての重要な点で基準回答と一貫している\n"
        "評価は数字のみで回答し、説明は付けないでください。\n\n"
        "質問: {query}\n"
        "基準回答: {sentence_inference}\n"
        "ユーザー回答: {sentence_true}\n"
        "評価:"
    )

    # 第3のテンプレート：より直接的で簡潔
    template_accuracy3 = (
        "質問に対する2つの回答を比較評価してください。\n"
        "ユーザー回答が基準回答の内容をどれだけ正確に反映しているかを0〜5で評価：\n"
        "0=まったく不正確、5=完全に一致\n\n"
        "質問: {query}\n"
        "ユーザー回答: {sentence_inference}\n"
        "基準回答: {sentence_true}\n"
        "評価点数（0〜5の数字のみ）:"
    )

    scores = []

    # テンプレート1での評価
    result1 = openai_generator(
        template_accuracy1.format(
            query=query,
            sentence_inference=response,
            sentence_true=reference
        )
    )
    score1 = extract_numeric_score(result1)
    scores.append(score1)

    # テンプレート2での評価（順序入れ替え）
    result2 = openai_generator(
        template_accuracy2.format(
            query=query,
            sentence_inference=reference,
            sentence_true=response
        )
    )
    score2 = extract_numeric_score(result2)
    scores.append(score2)

    # テンプレート3での評価（追加の視点）
    result3 = openai_generator(
        template_accuracy3.format(
            query=query,
            sentence_inference=response,
            sentence_true=reference
        )
    )
    score3 = extract_numeric_score(result3)
    scores.append(score3)

    # 外れ値を除外するため、中央値を使用
    scores.sort()
    if len(scores) >= 3:
        final_score = scores[1]  # 3つの値の中央値
    else:
        # スコアが不足している場合は平均を使用
        final_score = sum(scores) / len(scores) if scores else 0

    return final_score

In [13]:
# # 評価
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

1


## 結果 (ベースモデル)

Meta-Llama-3-8B-Instructは「Depseekショック」について誤った知識を提示しました：
* モデルはDeepSeekが特定のLLMモデルであることを理解（認識）しておらず、誤った回答を生成しました。

---

# 2. 文字起こしデータの活用
## 講義内容をソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: LLM講座第4講における講師の発言内容
* **目的**: モデルに「Depseekショック」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: 音声認識モデル(speech2text)で書き起こした生テキストをそのまま使用
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [14]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 8192

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [15]:
with open("/content/llm-engineering-RAG/lecture3/train_00.txt", "r") as f:
  raw_writedown = f.read()

In [16]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[100])

ドキュメントサイズ:  250
ドキュメントの例: 
 **2. はじめに**
表現学習は、データポイント間の類似性を構築し、その構造を反映する埋め込みを学習することにしばしば依拠する


In [17]:
# Retrievalの実行
# question = "LLMにおけるInference Time Scalingとは？"
question = "Deepseekショックとは、何かについて分かりやすく教えて。"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[59.026790618896484, 56.41819381713867, 56.97548294067383, 57.41816329956055, 54.140480041503906, 56.055259704589844, 55.433021545410156, 55.3321533203125, 55.3757438659668, 58.106468200683594, 57.3685417175293, 57.06796646118164, 56.76207733154297, 59.015159606933594, 57.53065490722656, 56.693138122558594, 58.98072814941406, 59.108890533447266, 59.08207321166992, 55.8441276550293, 58.50017547607422, 58.881317138671875, 56.733192443847656, 57.60691833496094, 57.75947570800781, 57.86536407470703, 55.51112747192383, 57.328094482421875, 58.82270431518555, 58.516944885253906, 56.795536041259766, 57.945884704589844, 58.14419937133789, 57.60190963745117, 54.17106246948242, 57.30091857910156, 54.656959533691406, 56.88519287109375, 57.178009033203125, 53.6611213684082, 55.537384033203125, 56.16973114013672, 54.63914108276367, 57.863502502441406, 59.26504898071289, 55.83369827270508, 55.614261627197266, 53.6168212890625, 54.890323638916016, 56.531368255615234, 57.35519027709961, 58.43843078613

In [18]:
topk = 10
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 80.19807434082031)
---


---

DeepSeekショックとは、中国浙江省杭州市を拠点とするAI開発企業が提供する次世代AI推論エンジン「DeepSeek」の登場により引き起こされた、業界全体の急激な変化とインパクトを指します 


取得したドキュメント2: (Score: 78.789306640625)
なぜDeepSeekショックが注目されるのか？
技術革新のタイミング
AI技術の進化が成熟段階に差し掛かりつつある中で、DeepSeekは「使いやすさ」と「即効性」という特長を兼ね備え、他の技術を凌駕する形で登場しました 


取得したドキュメント3: (Score: 77.43507385253906)
DeepSeekショックは単なる技術革新ではなく、データ主導型のビジネスモデルや意思決定プロセスを根底から変える現象として注目されています 


取得したドキュメント4: (Score: 76.97906494140625)
DeepSeekショックが市場にもたらす影響
業務効率化とコスト削減
DeepSeekの導入により、これまで人手や長時間を要していたデータ解析が短時間で行えるようになり、多くの企業がコスト削減と効率化を実現しています 


取得したドキュメント5: (Score: 73.14313507080078)
DeepSeekの3つの特徴
超高速なデータ処理 DeepSeekは、大量のデータをリアルタイムで解析する能力を持ち、迅速な意思決定を可能にします 


取得したドキュメント6: (Score: 71.9905776977539)
非技術者への普及
DeepSeekは、非技術者でも使いやすい設計となっているため、専門知識が無い方においても分析の敷居が下がる期待があります 


取得したドキュメント7: (Score: 71.79869079589844)
実績と信頼性
DeepSeekは既に複数の業界で導入され、その成果が報告されています 


取得したドキュメント8: (Score: 69.30848693847656)
市場のニーズとの一致
データ活用の重要性が高まる中で、DeepSeekの提供するソリューションは、多くの企業が直面する課題を解決するものとして期待されて

In [19]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
# question =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"

question =  f"[参考資料]\n{references}\n\n[質問] Deepseekショックとは、何かについて分かりやすく教えて。"

response = generate_output(question, system_prompt)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [20]:
print(response)

DeepSeekショックは、AI開発企業が提供する次世代AI推論エンジン「DeepSeek」の登場により引き起こされた、業界全体の急激な変化とインパクトを指します。DeepSeekは、AI技術の進化が成熟段階に差し掛かりつつある中で、「使いやすさ」と「即効性」という特長を兼ね備え、他の技術を凌駕する形で登場しました。

DeepSeekショックは、単なる技術革新ではなく、データ主導型のビジネスモデルや意思決定プロセスを根底から変える現象として注目されています。具体的には、DeepSeekの導入により、業務効率化とコスト削減が実現され、多くの企業が短時間で行えるようになるデータ解析が可能になりました。

DeepSeekショックが市場にもたらす影響として、業務効率化とコスト削減、超高速なデータ処理、非技術者への普及、実績と信頼性、市場のニーズとの一致など、多くの点で注目されています。


In [21]:
# 評価 (openai apikeyがある場合のみ実行)
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

4


## 結果 (修正テキストによるRAG)

書き起こしテキストの品質改善により、モデルの回答に部分的な向上が見られました：

### 改善点
* 「DeepSeekショックは、AI開発企業が提供する次世代AI推論エンジン「DeepSeek」の登場により引き起こされた、業界全体の急激な変化とインパクトを指します。」という事実を正確に捉えるようになった

### 問題点
* 特にない。直近の情報にも関わらず、高い精度で取得出来ている。

### 問題分析

モデルが誤った回答を生成する主要因として、**文脈の欠如**が考えられます：
* 「。」で区切られた短い文単位での検索では、各文の発言背景や関連性が失われる
* 単独の文から情報を抽出するため、講師の全体的な主張や議論の流れを把握できない
* 結果として、正しい個別の文でも、その解釈に必要な背景情報が欠如し、誤った文脈で理解される

# 3. 文脈を考慮したチャンク化の導入

検索結果の品質向上のため、以下の改善を実施します：

* **前後文脈を含むチャンク化**:
  - 検索でマッチした文だけでなく、その前後の複数文も含めてチャンクとして取得
  - 具体的には、マッチした文を中心に前2文、後2文を含む計5文程度のチャンクを構成
  - この「文脈ウィンドウ」により、発言の背景情報や議論の流れが保持される

* **期待される効果**:
  - 講師の主張とその根拠の関係性を正確に把握できる
  - 概念の定義とその適用範囲を正しく理解できる

この改善により、モデルが講義内容の本質をより正確に理解し、一貫性のある事実に基づいた回答を生成することが期待されます。

In [25]:
# references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])

# 前後それぞれ2つずつの文章を一つのドキュメントに追加する。（要は5つの文章集合になる)
references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] Deepseekショックとは、何かについて分かりやすく教えて。"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [26]:
print(response)

DeepSeekショックは、中国浙江省杭州市を拠点とするAI開発企業が提供する次世代AI推論エンジン「DeepSeek」の登場により引き起こされた、業界全体の急激な変化とインパクトを指します。この技術は、膨大なデータを短時間で解析し、適切な予測や意思決定を支援する能力を持つことで、従来の業務や市場構造に劇的な影響を与えています。

DeepSeekショックは、競争環境の激化や業務効率化、コスト削減など、多くの企業が直面する課題を解決するものとして期待されています。また、非技術者でも使いやすい設計となっているため、専門知識が無い方においても分析の敷居が下がる期待があります。

DeepSeekショックは、AI技術の進化が成熟段階に差し掛かりつつある中で、DeepSeekが「使いやすさ」と「即効性」という特長を兼ね備え、他の技術を凌駕する形で登場しました。実績と信


In [27]:
# 評価 (openai apikeyがある場合のみ実行)
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

5


## 結果 (文脈付きチャンク化によるRAG)

文脈を含むチャンク化により、モデルの回答の方向性に明確な改善が見られました：

### 改善点
* 「推論時の計算をスケールさせる」という概念を据えて回答
* Inference Time Scalingの基本原理についての理解が向上

### 残存する問題点
* 質問と関連性の低い情報（ノイズ）が混入する

### 問題分析

文脈付きチャンク化によるアプローチで新たに発生した課題：

1. **情報過多の問題**:
   * ドキュメント量の増加により、モデルに提供される情報総量が大幅に増加
   * 関連情報と非関連情報が混在し、ノイズと重要情報の区別が困難に

2. **情報選択の複雑化**:
   * モデルは単に回答を生成するだけでなく、提供された多様な情報源から関連性の高い情報を選別する作業も担うことになった
   * この二重タスクにより回答生成の難易度が上昇

# 4. Rerankによる情報品質の向上

検索精度をさらに向上させるため、二段階の検索プロセスを導入します：

* **Rerank手法の導入**:
  - 第一段階: 従来通り基本的な検索アルゴリズムでtop-k個のドキュメントチャンクを取得
  - 第二段階: 取得したチャンクに対してLLMを活用した高度な関連性評価を実施
  - LLMに「このドキュメントは質問『LLMにおけるInference Time Scalingとは？』に対して本当に関連性が高いか」を判断させる
  - 関連性スコアに基づいてランク付けし、真に関連性の高いチャンクのみを選出

* **期待される効果**:
  - 質の高い情報に焦点を絞ることで、ノイズとなる情報を大幅に削減
  - 文脈を保ちながらも、関連性の高い情報のみをモデルに提供
  - モデルのタスクを「多量の情報から選別して回答」から「厳選された情報に基づいて回答」へと単純化

この高度な情報フィルタリングにより、Inference Time Scalingに関する正確で一貫性のある回答生成が期待されます。

In [28]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = []
for ref in ["。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]]:

  system_prompt = "与えられた参考資料が質問に直接関連しているか？'yes''no'で答えること。ただし、余計なテキストを生成しないこと。"
  question =  f"[参考資料]\n{ref}\n\n[質問] Deepseekショックとは、何かについて分かりやすく教えて。"
  response = generate_output(question, system_prompt)

  print("\n\n対象となるドキュメント:\n", ref.replace("。", "。\n"))
  print("\n関連しているかどうか: ", response)

  if "yes" in response.lower():
    references.append(ref)

  print("\n\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.




対象となるドキュメント:
 たとえば、日本列島改造論を唱えた自民党の田中角栄は、地価や物価の高騰に加えて「田中金脈問題」の摘発などを受けて総辞職に追い込まれました*4）。
民主党の鳩山由紀夫は普天間基地移設問題や社民党の連立政権離脱の責任を取って総辞職しています5）。
---


---

DeepSeekショックとは、中国浙江省杭州市を拠点とするAI開発企業が提供する次世代AI推論エンジン「DeepSeek」の登場により引き起こされた、業界全体の急激な変化とインパクトを指します。
この技術は、膨大なデータを短時間で解析し、適切な予測や意思決定を支援する能力を持つことで、従来の業務や市場構造に劇的な影響を与えています

関連しているかどうか:  no





The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.




対象となるドキュメント:
 競争環境の激化
この技術がもたらす高速なデータ解析と精度の高い意思決定は、一部の先進的な企業に競争優位性をもたらし、業界全体での競争環境を劇的に変えています。
非技術者への普及
DeepSeekは、非技術者でも使いやすい設計となっているため、専門知識が無い方においても分析の敷居が下がる期待があります。
なぜDeepSeekショックが注目されるのか？
技術革新のタイミング
AI技術の進化が成熟段階に差し掛かりつつある中で、DeepSeekは「使いやすさ」と「即効性」という特長を兼ね備え、他の技術を凌駕する形で登場しました。
実績と信頼性
DeepSeekは既に複数の業界で導入され、その成果が報告されています

関連しているかどうか:  no





The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...



対象となるドキュメント:
 非技術者への普及
DeepSeekは、非技術者でも使いやすい設計となっているため、専門知識が無い方においても分析の敷居が下がる期待があります。
なぜDeepSeekショックが注目されるのか？
技術革新のタイミング
AI技術の進化が成熟段階に差し掛かりつつある中で、DeepSeekは「使いやすさ」と「即効性」という特長を兼ね備え、他の技術を凌駕する形で登場しました。
実績と信頼性
DeepSeekは既に複数の業界で導入され、その成果が報告されています。
特に、金融や医療といった精度が求められる分野での成功事例が、注目を集めています

関連しているかどうか:  yes





The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.




対象となるドキュメント:
 実績と信頼性
DeepSeekは既に複数の業界で導入され、その成果が報告されています。
特に、金融や医療といった精度が求められる分野での成功事例が、注目を集めています。
市場のニーズとの一致
データ活用の重要性が高まる中で、DeepSeekの提供するソリューションは、多くの企業が直面する課題を解決するものとして期待されています。
---


---
Anthropic の MCP の概要

1. はじめに
AIアシスタントが主流になるにつれて、業界はモデルに多額の投資を行い、推論と品質の急速な進歩を達成してきました

関連しているかどうか:  no





The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.




対象となるドキュメント:
 本研究は、フルファインチューニングが実行不可能な推論タスクのためのRLを可能にするという補完的な問題設定を対象としており、限られたハードウェアで作業する研究者や実務家にとって重要なギャップを埋める軽量かつ効率的なアプローチを提供します。
今後の作業では、これらの技術をよりセマンティックに多様なタスクに拡張し、トークンレベル最適化の効率ときめ細かい信用割り当ての利点を保持しつつスケールアップする戦略を研究する予定です。
---



---

**内容：Recursive KL Divergence Optimization (RKDO) に関する調査**

**1. 概要**
本報告書は、表現学習の新しい枠組みであるRecursive KL Divergence Optimization (RKDO) に関する論文「Recursive KL Divergence Optimization: A Dynamic Framework for Representation Learning」の抜粋に基づいており、その提案手法、実験結果、および理論的分析について記述する。
RKDOは、従来の表現学習手法を局所的な条件付き分布に対する再帰的なダイバージェンス整合プロセスとして再構築することで一般化するものである

関連しているかどうか:  no





対象となるドキュメント:
 全てのデータセットと訓練期間において、RKDOはI-Conと比較して**約30%低い損失値**を一貫して達成した（27%から34%の改善）。
これらの改善は統計的に有意である (p < 0.001)。
これは、RKDOの再帰的な更新メカニズムが表現学習のための根本的により効率的な最適化ランドスケープを創出することを示唆している。
**5.2. 計算資源効率: 訓練時間の60-80%削減**
RKDOは顕著な早期エポック性能を示し、しばしばI-Conのより長い訓練と同等の結果を達成するために**60-80%少ない計算資源**（訓練エポック）しか必要としない

関連しているかどうか:  no





In [29]:
print(len(references))

4


In [30]:
print(references)

['---\n\n\n---\n\nDeepSeekショックとは、中国浙江省杭州市を拠点とするAI開発企業が提供する次世代AI推論エンジン「DeepSeek」の登場により引き起こされた、業界全体の急激な変化とインパクトを指します。この技術は、膨大なデータを短時間で解析し、適切な予測や意思決定を支援する能力を持つことで、従来の業務や市場構造に劇的な影響を与えています。DeepSeekショックは単なる技術革新ではなく、データ主導型のビジネスモデルや意思決定プロセスを根底から変える現象として注目されています。DeepSeekの3つの特徴\n超高速なデータ処理 DeepSeekは、大量のデータをリアルタイムで解析する能力を持ち、迅速な意思決定を可能にします', 'この技術は、膨大なデータを短時間で解析し、適切な予測や意思決定を支援する能力を持つことで、従来の業務や市場構造に劇的な影響を与えています。DeepSeekショックは単なる技術革新ではなく、データ主導型のビジネスモデルや意思決定プロセスを根底から変える現象として注目されています。DeepSeekの3つの特徴\n超高速なデータ処理 DeepSeekは、大量のデータをリアルタイムで解析する能力を持ち、迅速な意思決定を可能にします。汎用性の高さ 金融、医療、製造、マーケティングなど、多様な分野に応用可能な柔軟性を持っています', 'DeepSeekショックが市場にもたらす影響\n業務効率化とコスト削減\nDeepSeekの導入により、これまで人手や長時間を要していたデータ解析が短時間で行えるようになり、多くの企業がコスト削減と効率化を実現しています。競争環境の激化\nこの技術がもたらす高速なデータ解析と精度の高い意思決定は、一部の先進的な企業に競争優位性をもたらし、業界全体での競争環境を劇的に変えています。非技術者への普及\nDeepSeekは、非技術者でも使いやすい設計となっているため、専門知識が無い方においても分析の敷居が下がる期待があります。なぜDeepSeekショックが注目されるのか？\n技術革新のタイミング\nAI技術の進化が成熟段階に差し掛かりつつある中で、DeepSeekは「使いやすさ」と「即効性」という特長を兼ね備え、他の技術を凌駕する形で登場しました', '非技術者への普及\nDeepSeekは、非技術者

上記より、上位3件のみが関連しているとわかったので、これらだけをモデルに渡すこととする。

In [31]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] Deepseekショックとは、何かについて分かりやすく教えて。"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [32]:
print(response)

DeepSeekショックは、中国のAI開発企業が提供するAI推論エンジン「DeepSeek」が登場したことで、業界全体に大きな影響を与える現象です。DeepSeekは、膨大なデータを短時間で解析し、適切な予測や意思決定を支援する能力を持っています。

この技術は、従来の業務や市場構造に劇的な影響を与えることで、業界全体に大きな変化をもたらします。DeepSeekショックは、データ主導型のビジネスモデルや意思決定プロセスを根底から変える現象として注目されています。

DeepSeekの3つの特徴は、超高速なデータ処理、汎用性の高さ、実績と信頼性です。超高速なデータ処理能力を持つことで、迅速な意思決定を可能にします。汎用性の高さを持っていますため、多様な分野に応用可能です。実績と信頼性を持っていますため、既に複数の業界で導入され、その成果が報告されています。

Deep


In [33]:
# 評価 (openai apikeyがある場合のみ実行)
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

4


## 結果 (Rerank導入後)

Rerankの導入により、回答品質に改善が見られました：

### 達成された成果
* Inference Time Scalingに関する正確な情報を含んだ回答の生成
* 無関係な情報やノイズの排除
* 講義内容を反映した説明の実現 🎉

この結果から、RAGパイプラインにおける情報の質と関連性の重要性であり、検索で取得した情報を単に増やすだけでなく、その情報の関連性を精査する方法を学ぶことができました。

---


# 5. さらなる改善案: 意味的チャンク化

文単位での分割と前後文脈の追加という現在のアプローチをさらに発展させる手法として、**意味的なチャンク化**が考えられます：

* **意味的チャンク（段落）単位での分割**:
  - 単純な文の区切りではなく、意味的なまとまり（トピック、議論、例示など）に基づいてテキストを分割
  - 人間の主観に基づく意味的な段落分けを活用
  - 各チャンクが「一つの完結した考え」を表現するようにする

* **期待される効果**:
  - より自然な文脈理解が可能に（人間の思考や会話の流れに近い）
  - トピックの開始から結論までの流れを維持できる
  - 概念間の関係性や比較が同一チャンク内に含まれ、より深い理解につなげる

* **検証方法**:
  - 人間が主観的に意味でグループ化したチャンクセットを用意
  - 同じRerank手法を適用し、文単位チャンクとの性能差を比較
  - 回答の正確性、一貫性、網羅性を評価指標として使用

この意味的チャンク化手法は、特に講義のような構造化された発話においては、より自然で効果的な情報検索と理解を可能にすると予想されます。

In [34]:
# 本来は段落をそのままdocumentsに入れずに一定のサイズに分割した方が良いでしょうが、簡単のために段落をそのまま入れてしまいます。
documents = [text.replace("\n", " ").strip() for text in raw_writedown.split("\n\n")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[1])

ドキュメントサイズ:  112
ドキュメントの例: 
 **著者：** Alan Lee, Harry Tong (ミシガン大学 計算機科学科)


In [35]:
question = "Deepseekショックとは、何かについて分かりやすく教えて。"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[58.98407745361328, 57.54206085205078, 58.589256286621094, 57.118045806884766, 54.193885803222656, 55.644065856933594, 55.04263687133789, 57.710594177246094, 56.275238037109375, 57.02692794799805, 56.78331756591797, 58.759849548339844, 56.72895812988281, 58.63357162475586, 58.12657928466797, 60.020416259765625, 57.933658599853516, 55.93062210083008, 57.21714401245117, 55.00763702392578, 58.87317657470703, 56.399757385253906, 56.30885314941406, 54.52102279663086, 56.657073974609375, 54.49985122680664, 55.68645095825195, 58.03173828125, 58.28377151489258, 56.49665451049805, 55.806053161621094, 56.02516555786133, 55.89545822143555, 55.92705154418945, 56.111328125, 58.04477310180664, 55.18482971191406, 55.97125244140625, 57.59028625488281, 53.61223220825195, 55.87009048461914, 58.58213806152344, 57.340858459472656, 57.128292083740234, 57.116050720214844, 57.37501525878906, 57.77354431152344, 59.77959442138672, 57.75252151489258, 56.393985748291016, 57.67215347290039, 58.057891845703125, 5

In [36]:
# 簡単のためにtop2でやります。結果を見てもらえれば問題なく関連する項目のみ取得できているのが分かるかと思います。
topk = 10
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 80.11436462402344)
DeepSeekショックとは、中国浙江省杭州市を拠点とするAI開発企業が提供する次世代AI推論エンジン「DeepSeek」の登場により引き起こされた、業界全体の急激な変化とインパクトを指します。この技術は、膨大なデータを短時間で解析し、適切な予測や意思決定を支援する能力を持つことで、従来の業務や市場構造に劇的な影響を与えています。 DeepSeekショックは単なる技術革新ではなく、データ主導型のビジネスモデルや意思決定プロセスを根底から変える現象として注目されています。 


取得したドキュメント2: (Score: 77.57918548583984)
なぜDeepSeekショックが注目されるのか？ 技術革新のタイミング AI技術の進化が成熟段階に差し掛かりつつある中で、DeepSeekは「使いやすさ」と「即効性」という特長を兼ね備え、他の技術を凌駕する形で登場しました。 


取得したドキュメント3: (Score: 76.81721496582031)
DeepSeekショックが市場にもたらす影響 業務効率化とコスト削減 DeepSeekの導入により、これまで人手や長時間を要していたデータ解析が短時間で行えるようになり、多くの企業がコスト削減と効率化を実現しています。 


取得したドキュメント4: (Score: 73.31060028076172)
DeepSeekの3つの特徴 超高速なデータ処理 DeepSeekは、大量のデータをリアルタイムで解析する能力を持ち、迅速な意思決定を可能にします。 


取得したドキュメント5: (Score: 71.24407958984375)
非技術者への普及 DeepSeekは、非技術者でも使いやすい設計となっているため、専門知識が無い方においても分析の敷居が下がる期待があります。 


取得したドキュメント6: (Score: 70.50711059570312)
実績と信頼性 DeepSeekは既に複数の業界で導入され、その成果が報告されています。特に、金融や医療といった精度が求められる分野での成功事例が、注目を集めています。 


取得したドキュメント7: (Score: 69.03985595703125)
市場のニーズとの一致 

In [37]:
reference = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] Deepseekショックとは、何かについて分かりやすく教えて。"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [38]:
print(response)

DeepSeekショックは、中国のAI開発企業が提供するAI推論エンジン「DeepSeek」が登場したことで、業界全体に大きな影響を与える現象です。DeepSeekは、膨大なデータを短時間で解析し、適切な予測や意思決定を支援する能力を持っています。

この技術は、従来の業務や市場構造に劇的な影響を与えることで、業界全体に大きな変化をもたらします。DeepSeekショックは、データ主導型のビジネスモデルや意思決定プロセスを根底から変える現象として注目されています。

DeepSeekの3つの特徴は、超高速なデータ処理、汎用性の高さ、実績と信頼性です。超高速なデータ処理能力を持つことで、迅速な意思決定を可能にします。汎用性の高さを持っていますため、多様な分野に応用可能です。実績と信頼性を持っていますため、既に複数の業界で導入され、その成果が報告されています。

Deep


In [39]:
# 評価 (openai apikeyがある場合のみ実行)
score = evaluate_answer_accuracy(question, response, gold_answer)
print(score)

4
